In [ ]:
# Cell 1: Optimal Package Installation
print("Installing packages with optimal configuration...")

# Uninstall conflicts
!pip uninstall -y torch transformers huggingface-hub sentence-transformers peft -q 2>/dev/null

# Install core packages (optimized order)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers huggingface-hub sentence-transformers peft
!pip install -q datasets rouge-score mlflow pandas numpy scikit-learn
!pip install -q google-cloud-storage matplotlib seaborn plotly

# Verify installation
import torch
import transformers

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ Transformers: {transformers.__version__}")
print(f"✓ GPU: {'YES ✅ ' + torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO ❌ (CPU)'}")
if torch.cuda.is_available():
    print(f"✓ Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print("="*60)

Installing packages with optimal configuration...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.22 requires huggingface_hub, which is not installed.
accelerate 1.11.0 requires huggingface_hub>=0.21.0, which is not installed.
torchtune 0.6.1 requires huggingface_hub[hf_transfer], which is not installed.

SETUP COMPLETE
✓ PyTorch: 2.7.1+cu118
✓ Transformers: 4.57.1
✓ GPU: YES ✅ Tesla T4
✓ Memory: 14.7 GB


In [ ]:
# Cell 2: GCP Authentication
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = "regal-bonito-455919-u3"
BUCKET_NAME = "lab-lens-data-regal-bonito-455919-u3"

!gcloud config set project {PROJECT_ID}

print("="*60)
print("GCP AUTHENTICATION")
print("="*60)
print(f"✅ Project: {PROJECT_ID}")
print(f"✅ Bucket: gs://{BUCKET_NAME}")
print("="*60)

INFORMATION: Project 'regal-bonito-455919-u3' has no 'environment' tag set. Use either 'Production', 'Development', 'Test', or 'Staging'. Add an 'environment' tag using `gcloud resource-manager tags bindings create`.
Updated property [core/project].
GCP AUTHENTICATION
✅ Project: regal-bonito-455919-u3
✅ Bucket: gs://lab-lens-data-regal-bonito-455919-u3


In [ ]:
# Cell 3: Download ENHANCED data from GCP
print("="*60)
print("DOWNLOADING ENHANCED DATA (WITH GEMINI NARRATIVES)")
print("="*60)

# Remove old data if exists
!rm -f train.csv validation.csv test.csv

# Download new enhanced datasets
!gsutil -m cp gs://{BUCKET_NAME}/data/model_ready/train.csv .
!gsutil -m cp gs://{BUCKET_NAME}/data/model_ready/validation.csv .
!gsutil -m cp gs://{BUCKET_NAME}/data/model_ready/test.csv .

import os
print("\n✅ Downloaded files:")
for file in ['train.csv', 'validation.csv', 'test.csv']:
    size_mb = os.path.getsize(file) / 1024 / 1024
    print(f"  {file}: {size_mb:.2f} MB")
print("="*60)

DOWNLOADING ENHANCED DATA (WITH GEMINI NARRATIVES)
Copying gs://lab-lens-data-regal-bonito-455919-u3/data/model_ready/train.csv...
\ [1/1 files][ 50.4 MiB/ 50.4 MiB] 100% Done                                    
Operation completed over 1 objects/50.4 MiB.                                     
Copying gs://lab-lens-data-regal-bonito-455919-u3/data/model_ready/validation.csv...
- [1/1 files][ 11.1 MiB/ 11.1 MiB] 100% Done                                    
Operation completed over 1 objects/11.1 MiB.                                     
Copying gs://lab-lens-data-regal-bonito-455919-u3/data/model_ready/test.csv...
- [1/1 files][ 10.9 MiB/ 10.9 MiB] 100% Done                                    
Operation completed over 1 objects/10.9 MiB.                                     

✅ Downloaded files:
  train.csv: 50.40 MB
  validation.csv: 11.12 MB
  test.csv: 10.90 MB


In [ ]:
# Cell 4: Data Quality Validation (Simple & Effective)
import pandas as pd

print("="*60)
print("DATA QUALITY VALIDATION")
print("="*60)

# Load data
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('validation.csv')
test_df = pd.read_csv('test.csv')

print(f"Train: {len(train_df)} records")
print(f"Validation: {len(val_df)} records")
print(f"Test: {len(test_df)} records")
print(f"Total columns: {len(train_df.columns)}")

# Validation checks
print("\n" + "="*60)
print("VALIDATION CHECKS")
print("="*60)

validation_results = {}

# Check 1: Required columns exist
required_cols = ['input_text', 'target_summary', 'age_group', 'gender', 'ethnicity_clean']
for col in required_cols:
    exists = col in train_df.columns
    validation_results[f'column_{col}_exists'] = exists
    print(f"✅ Column '{col}' exists: {exists}")

# Check 2: No null values in critical columns
null_check_input = train_df['input_text'].isna().sum() == 0
null_check_target = train_df['target_summary'].isna().sum() == 0
validation_results['no_nulls_input'] = null_check_input
validation_results['no_nulls_target'] = null_check_target
print(f"✅ No nulls in 'input_text': {null_check_input}")
print(f"✅ No nulls in 'target_summary': {null_check_target}")

# Check 3: Text length ranges
input_min = train_df['input_length'].min()
input_max = train_df['input_length'].max()
summary_min = train_df['summary_length'].min()
summary_max = train_df['summary_length'].max()

input_valid = (input_min >= 100) and (input_max <= 50000)
summary_valid = (summary_min >= 500) and (summary_max <= 2000)
validation_results['input_length_valid'] = input_valid
validation_results['summary_length_valid'] = summary_valid

print(f"✅ Input length range: {input_min}-{input_max} chars (valid: {input_valid})")
print(f"✅ Summary length range: {summary_min}-{summary_max} chars (valid: {summary_valid})")

# Check 4: Demographic distribution
print(f"\nDemographic Distribution:")
for demo_col in ['age_group', 'gender', 'ethnicity_clean']:
    if demo_col in train_df.columns:
        print(f"  {demo_col}: {train_df[demo_col].value_counts().to_dict()}")

# Overall validation
all_passed = all(validation_results.values())
print(f"\n{'='*60}")
print(f"VALIDATION STATUS: {'✅ ALL PASSED' if all_passed else '❌ SOME FAILED'}")
print(f"  Checks passed: {sum(validation_results.values())}/{len(validation_results)}")
print(f"{'='*60}")

# Show sample enhanced summary
print(f"\nSAMPLE ENHANCED SUMMARY (WITH GEMINI):")
print(f"{'='*60}")
print(f"Input length: {len(train_df.iloc[0]['input_text'])} chars")
print(f"Summary length: {len(train_df.iloc[0]['target_summary'])} chars")
print(f"\nEnhanced Summary:\n{train_df.iloc[0]['target_summary']}")
print("="*60)

DATA QUALITY VALIDATION
Train: 1638 records
Validation: 351 records
Test: 351 records
Total columns: 51

VALIDATION CHECKS
✅ Column 'input_text' exists: True
✅ Column 'target_summary' exists: True
✅ Column 'age_group' exists: True
✅ Column 'gender' exists: True
✅ Column 'ethnicity_clean' exists: True
✅ No nulls in 'input_text': True
✅ No nulls in 'target_summary': True
✅ Input length range: 155-36758 chars (valid: True)
✅ Summary length range: 483-1465 chars (valid: False)

Demographic Distribution:
  age_group: {'65+': 730, '50-65': 472, '35-50': 220, '<18': 116, '18-35': 100}
  gender: {'M': 898, 'F': 740}
  ethnicity_clean: {'WHITE': 1145, 'OTHER': 234, 'BLACK': 166, 'HISPANIC': 57, 'ASIAN': 36}

VALIDATION STATUS: ❌ SOME FAILED
  Checks passed: 8/9

SAMPLE ENHANCED SUMMARY (WITH GEMINI):
Input length: 11707 chars
Summary length: 962 chars

Enhanced Summary:

PATIENT: 86.0-year-old M

DATES: Admitted 2199-08-23, Discharged 2199-09-01

ADMISSION: anemia work-up

HISTORY: 1. Coronary 

In [ ]:
# Cell 5: Setup MLflow
import mlflow
mlflow.set_tracking_uri("/content/mlruns")
mlflow.set_experiment("biobart_enhanced_training")
print("✅ MLflow initialized")

# Cell 6: Load BioBART Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

MODEL_NAME = "GanjinZero/biobart-v2-base"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
print(f"✅ Model loaded on {device.upper()}")

# Cell 7: Create Dataset
from torch.utils.data import Dataset

class MIMICSummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_length=512, max_target_length=256):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_encoding = self.tokenizer(str(row['input_text']), max_length=self.max_input_length, padding='max_length', truncation=True, return_tensors='pt')
        target_encoding = self.tokenizer(str(row['target_summary']), max_length=self.max_target_length, padding='max_length', truncation=True, return_tensors='pt')
        return {'input_ids': input_encoding['input_ids'].flatten(), 'attention_mask': input_encoding['attention_mask'].flatten(), 'labels': target_encoding['input_ids'].flatten()}

train_dataset = MIMICSummarizationDataset(train_df, tokenizer)
val_dataset = MIMICSummarizationDataset(val_df, tokenizer)
test_dataset = MIMICSummarizationDataset(test_df, tokenizer)
print(f"✅ Datasets created")

# Cell 8: Configure Training
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

OUTPUT_DIR = "/content/biobart_enhanced"
mlflow.start_run(run_name="biobart_gemini_enhanced")
mlflow.log_params({'train_samples': len(train_dataset), 'model': MODEL_NAME, 'enhancement': 'gemini_api'})

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR, num_train_epochs=3, per_device_train_batch_size=4,
    per_device_eval_batch_size=4, learning_rate=2e-5, warmup_steps=100,
    eval_strategy="steps", eval_steps=200, save_strategy="steps", save_steps=200,
    save_total_limit=2, load_best_model_at_end=True, metric_for_best_model='eval_loss',
    predict_with_generate=True, generation_max_length=256, fp16=True,
    logging_steps=50, report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
trainer = Seq2SeqTrainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset, tokenizer=tokenizer, data_collator=data_collator)
print("✅ Training configured")

/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/19 01:02:32 INFO mlflow.tracking.fluent: Experiment with name 'biobart_enhanced_training' does not exist. Creating a new experiment.


✅ MLflow initialized


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/666M [00:00<?, ?B/s]

✅ Model loaded on CUDA
✅ Datasets created
✅ Training configured


/tmp/ipython-input-4241367491.py:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset, tokenizer=tokenizer, data_collator=data_collator)


In [ ]:
# Cell 9: Train Model
import time
print("="*60)
print("TRAINING ENHANCED MODEL")
print("="*60)
start = time.time()
train_result = trainer.train()
duration = time.time() - start
mlflow.log_metrics({'train_loss': train_result.training_loss, 'duration_min': duration/60})
print(f"✅ Training complete: {duration/60:.1f} min")
print(f"✅ Final loss: {train_result.training_loss:.4f}")

TRAINING ENHANCED MODEL


/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
200,1.846400,1.648427
400,1.619100,1.429057
600,1.415200,1.355302
800,1.403300,1.314757
1000,1.282800,1.296320
1200,1.317800,1.288203


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


✅ Training complete: 9.0 min
✅ Final loss: 1.7493


In [ ]:
# Cell 10: Comprehensive Evaluation with Sample Generation
from rouge_score import rouge_scorer
import numpy as np

print("="*60)
print("COMPREHENSIVE MODEL EVALUATION")
print("="*60)

# Part 1: Overall Test Set Evaluation
print("\n1. OVERALL TEST SET EVALUATION")
print("-"*60)
eval_results = trainer.evaluate(eval_dataset=test_dataset)

print(f"Test Set Metrics:")
print(f"  Loss: {eval_results['eval_loss']:.4f}")
print(f"  Runtime: {eval_results['eval_runtime']:.1f} seconds")
print(f"  Samples/second: {eval_results['eval_samples_per_second']:.2f}")

# Part 2: Calculate ROUGE Scores
print("\n2. ROUGE SCORE CALCULATION (50 samples)")
print("-"*60)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

# Generate summaries for 50 test samples
for idx in range(min(50, len(test_df))):
    row = test_df.iloc[idx]

    # Tokenize and generate
    inputs = tokenizer(
        str(row['input_text']),
        max_length=512,
        truncation=True,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reference = str(row['target_summary'])

    # Calculate ROUGE
    scores = scorer.score(reference, generated)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Average scores
avg_rouge1 = np.mean(rouge_scores['rouge1'])
avg_rouge2 = np.mean(rouge_scores['rouge2'])
avg_rougeL = np.mean(rouge_scores['rougeL'])

print(f"\nROUGE Scores (n=50):")
print(f"  ROUGE-1: {avg_rouge1:.4f}")
print(f"  ROUGE-2: {avg_rouge2:.4f}")
print(f"  ROUGE-L: {avg_rougeL:.4f}")

# Part 3: Generate and Display Detailed Sample Summaries
print("\n" + "="*60)
print("3. SAMPLE SUMMARIES (5 Diverse Examples)")
print("="*60)

# Select diverse samples
sample_indices = [0, 25, 50, 75, 100]
generated_samples = []

for i, idx in enumerate(sample_indices, 1):
    if idx >= len(test_df):
        continue

    row = test_df.iloc[idx]

    # Generate summary
    inputs = tokenizer(
        str(row['input_text']),
        max_length=512,
        truncation=True,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            length_penalty=1.0
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reference = str(row['target_summary'])

    # Calculate ROUGE for this sample
    sample_scores = scorer.score(reference, generated)

    # Display
    print(f"\n{'='*70}")
    print(f"SAMPLE {i}")
    print(f"{'='*70}")
    print(f"Demographics: {row.get('age_group', 'N/A')}, {row.get('gender', 'N/A')}, {row.get('ethnicity_clean', 'N/A')}")
    print(f"Input Length: {len(row['input_text'])} chars")
    print(f"Compression: {len(row['input_text'])}/{len(generated)} = {len(row['input_text'])/len(generated):.1f}x")
    print(f"ROUGE-L: {sample_scores['rougeL'].fmeasure:.4f}")

    print(f"\n{'─'*70}")
    print("REFERENCE SUMMARY (Gemini-enhanced target):")
    print(f"{'─'*70}")
    print(reference)

    print(f"\n{'─'*70}")
    print("MODEL GENERATED SUMMARY:")
    print(f"{'─'*70}")
    print(generated)
    print(f"{'='*70}")

    # Store for later
    generated_samples.append({
        'sample_id': i,
        'demographics': f"{row.get('age_group', 'N/A')}, {row.get('gender', 'N/A')}, {row.get('ethnicity_clean', 'N/A')}",
        'input_length': len(row['input_text']),
        'reference': reference,
        'generated': generated,
        'rouge_l': sample_scores['rougeL'].fmeasure
    })

# Part 4: Summary Statistics
print("\n" + "="*60)
print("4. EVALUATION SUMMARY")
print("="*60)
print(f"✅ Test Loss: {eval_results['eval_loss']:.4f}")
print(f"✅ ROUGE-1: {avg_rouge1:.4f}")
print(f"✅ ROUGE-2: {avg_rouge2:.4f}")
print(f"✅ ROUGE-L: {avg_rougeL:.4f}")
print(f"✅ Samples Generated: {len(generated_samples)}")
print(f"✅ Avg Sample ROUGE-L: {np.mean([s['rouge_l'] for s in generated_samples]):.4f}")
print("="*60)

# Save samples to JSON
import json
with open('/content/generated_samples_detailed.json', 'w') as f:
    json.dump(generated_samples, f, indent=2)

print("\n✅ Detailed samples saved: /content/generated_samples_detailed.json")
print("="*60)

COMPREHENSIVE MODEL EVALUATION

1. OVERALL TEST SET EVALUATION
------------------------------------------------------------


Test Set Metrics:
  Loss: 1.3023
  Runtime: 7.3 seconds
  Samples/second: 47.79

2. ROUGE SCORE CALCULATION (50 samples)
------------------------------------------------------------

ROUGE Scores (n=50):
  ROUGE-1: 0.3165
  ROUGE-2: 0.1531
  ROUGE-L: 0.2497

3. SAMPLE SUMMARIES (5 Diverse Examples)

SAMPLE 1
Demographics: <18, F, WHITE
Input Length: 7036 chars
Compression: 7036/708 = 9.9x
ROUGE-L: 0.1891

──────────────────────────────────────────────────────────────────────
REFERENCE SUMMARY (Gemini-enhanced target):
──────────────────────────────────────────────────────────────────────

PATIENT: 0.0-year-old F

DATES: Admitted 2198-04-12, Discharged 2198-05-23

ADMISSION: confusion

HISTORY: Past Oncologic History: Primarily taken from Dr. notes
form .
- Initially presented with melanoma in...

DIAGNOSIS: seizures due to metastatic melanoma

HOSPITAL COURSE: BRIEF HOSPITAL COURSE:
Ms.  is a 46y/o lady who has metastatic melanoma with brain
lesions, edema, and midline shift and presen

In [ ]:
# Save model
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Upload to GCP
!gsutil -m cp -r {OUTPUT_DIR}/* gs://{BUCKET_NAME}/models/biobart_enhanced/
!gsutil -m cp -r /content/mlruns gs://{BUCKET_NAME}/logs/

mlflow.end_run()
print(f"✅ Model saved to GCP: gs://{BUCKET_NAME}/models/biobart_enhanced/")

Copying file:///content/biobart_enhanced/checkpoint-1200/model.safetensors [Content-Type=application/octet-stream]...
Copying file:///content/biobart_enhanced/checkpoint-1200/training_args.bin [Content-Type=application/octet-stream]...
Copying file:///content/biobart_enhanced/checkpoint-1200/trainer_state.json [Content-Type=application/json]...
Copying file:///content/biobart_enhanced/checkpoint-1200/optimizer.pt [Content-Type=application/vnd.snesdev-page-table]...
Copying file:///content/biobart_enhanced/checkpoint-1200/tokenizer_config.json [Content-Type=application/json]...
Copying file:///content/biobart_enhanced/checkpoint-1200/tokenizer.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be 

In [ ]:
# Generate Sample Summaries from Trained Model
import torch

print("="*60)
print("GENERATING SAMPLE SUMMARIES")
print("="*60)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Select 5 diverse samples from test set
sample_indices = [0, 50, 100, 150, 200]

for i, idx in enumerate(sample_indices, 1):
    if idx >= len(test_df):
        continue

    row = test_df.iloc[idx]

    print(f"\n{'='*70}")
    print(f"SAMPLE {i}")
    print(f"{'='*70}")
    print(f"Demographics: {row.get('age_group', 'N/A')}, {row.get('gender', 'N/A')}, {row.get('ethnicity_clean', 'N/A')}")
    print(f"Input: {len(row['input_text'])} chars")

    # Generate summary
    inputs = tokenizer(
        str(row['input_text']),
        max_length=512,
        truncation=True,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            length_penalty=1.0
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"\n{'─'*70}")
    print("REFERENCE SUMMARY (Gemini-generated target):")
    print(f"{'─'*70}")
    print(row['target_summary'])

    print(f"\n{'─'*70}")
    print("MODEL GENERATED SUMMARY:")
    print(f"{'─'*70}")
    print(generated)

    print(f"\n{'─'*70}")
    print(f"Compression: {len(row['input_text'])}/{len(generated)} = {len(row['input_text'])/len(generated):.1f}x")
    print(f"{'='*70}\n")

print("\n" + "="*60)
print("✅ SAMPLE GENERATION COMPLETE")
print("="*60)

GENERATING SAMPLE SUMMARIES

SAMPLE 1
Demographics: <18, F, WHITE
Input: 7036 chars

──────────────────────────────────────────────────────────────────────
REFERENCE SUMMARY (Gemini-generated target):
──────────────────────────────────────────────────────────────────────

PATIENT: 0.0-year-old F

DATES: Admitted 2198-04-12, Discharged 2198-05-23

ADMISSION: confusion

HISTORY: Past Oncologic History: Primarily taken from Dr. notes
form .
- Initially presented with melanoma in...

DIAGNOSIS: seizures due to metastatic melanoma

HOSPITAL COURSE: BRIEF HOSPITAL COURSE:
Ms.  is a 46y/o lady who has metastatic melanoma with brain
lesions, edema, and midline shift and presented after seizures
with...

LABS: Potassium: 4.2 mEq/L;  Sodium: 139 mEq/L;  Triglycerides: 112 mg/dL

MEDICATIONS: 1. lamotrigine 100 mg Tablet Sig: 1.5 Tablets PO BID (2 times a
day).

FOLLOW-UP: Home With Service

CLINICAL SUMMARY: A 0.0-year-old female was admitted with confusion. Primary diagnoses included seizures d